In [2]:
import requests
import time
from datetime import timedelta
from datetime import datetime
import pandas as pd
import urllib.request
from tqdm import tqdm
import json
import asyncio
import numpy as np
import os
from nltk import sent_tokenize
from nltk import word_tokenize

In [47]:
def scrape_and_save_data(pageno):
    city = []
    state = []
    date = []
    lccn = []
    text = []
    title = []

    try:
        r = s.get(f'https://chroniclingamerica.loc.gov/search/pages/results/?andtext=coolie&format=json&page={pageno}', timeout=5)
        r.raise_for_status()  # Check if the request was successful
        data = r.json()

        for item in data.get('items', []):
            city.append(item.get('city', np.nan))
            date.append(item.get('date', np.nan))
            state.append(item.get('state', np.nan))
            lccn.append(item.get('lccn', np.nan))
            text.append(item.get('ocr_eng', np.nan))
            title.append(item.get('title', np.nan))

        df = pd.DataFrame({
            'city': city,
            'state': state,
            'date': date,
            'lCCN': lccn,
            'text': text,
            'title': title
        })

        output_dir =  '/Users/mstudio/Library/CloudStorage/Box-Box/coolie/raw-data/'
        os.makedirs(output_dir, exist_ok=True)
        df.to_csv(f'{output_dir}coolie_{pageno}.csv', index=False)
        time.sleep(30)
    except requests.exceptions.RequestException as e:
        # print(f"Error: {e}")
        failed_page.append(pageno)

failed_page = []
s = requests.Session()

for pageno in tqdm(range(int(np.round(124511 / 20)) + 1)): #6226
    no = pageno+1
    if str(no) in [i.strip('coolie_').strip('.csv') for i in os.listdir('/Users/mstudio/Library/CloudStorage/Box-Box/coolie/raw-data/')]:
        continue
    else:
        scrape_and_save_data(no)


100%|██████████| 6227/6227 [3:53:49<00:00,  2.25s/it]  


In [11]:
def keyword_search(df: pd.DataFrame, substring: str):
    state = []
    city = []
    date = []
    lccn = []
    sent_list = []
    title = []
    context_list = []

    for idx, val in df.iterrows():
        tokenized_sent = sent_tokenize(val['text'].lower())
        matching_items = [item for item in tokenized_sent if substring in item]
        to_context = []
        for num, sent in enumerate(tokenized_sent):
            interim_to_context = []
            if substring in sent:
                interim_to_context.extend(tokenized_sent[max(0, num - 2): num + 3])
                to_context.append(interim_to_context)

        state.append(val['state'])
        city.append(val['city'])
        date.append(val['date'])
        lccn.append(val['lCCN'])
        title.append(val['title'])
        sent_list.append(matching_items)
        context_list.append(to_context)

    newdf = pd.DataFrame({'state': state, 'city': city, 'date': date, 'lccn': lccn, 'sent': sent_list, 'title': title, 'context': context_list})
    return newdf

In [24]:
path='/Users/mstudio/Library/CloudStorage/Box-Box/coolie/raw-data/'
append_df=[]
for i in tqdm(os.listdir(path)):
    if i.startswith('._'):
        continue
    elif i.endswith('.csv'):
        df=pd.read_csv(path+i)
        df['text']=df['text'].astype(str)
        newdf=keyword_search(df, 'coolie')
        lendf=newdf[newdf['sent'].apply(lambda x: len(x) > 0)]
        append_df.append(lendf)


100%|██████████| 5499/5499 [08:55<00:00, 10.27it/s]


In [26]:
cooliedf=pd.concat(append_df).reset_index(drop=True)
cooliedf=cooliedf.explode(['sent', 'context'], ignore_index=True)
# cooliedf['lccn']=cooliedf['lccn'].astype(str)

In [33]:
cooliedf.head(4)

,state,city,date,lccn,sent,title,context
0,['New York'],['New York'],18910826,sn83030193,cholera breaks out on a steamer\nladen with oh...,The evening world. [volume],"[sessions., m m\ncruise oe a death ship., chol..."
1,['New York'],['New York'],18910826,sn83030193,atlt ices from\nslngnporo state that tho steam...,The evening world. [volume],"[san i'iuxcisco, aug., !'!!., atlt ices from\n..."
2,['Montana'],['Butte'],18830407,sn84036033,"therefore, it is not\nsurprising that the auth...",The semi-weekly miner. [volume],[at present the miserable\nkanakas are but one...
3,['Montana'],['Butte'],18830407,sn84036033,kanakas want coolies.,The semi-weekly miner. [volume],"[great excitement prevails, and it., is\nbelie..."


In [29]:
cooliedf.to_csv(path+'../'+'/sent/'+'coolie.csv', index=False)

In [34]:
cooliedf.shape

(118054, 7)